## Notebook #5: Final Kaggle Submission

In this notebook, I will present my Kaggle submission that scored the best. It is definitely not the model that worked best for my problem statement; however, it managed to have a lower RMSE and therefore did well in the scoring algorithm Kaggle subscribes to.

In [138]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [139]:
df_house = pd.read_csv('./datasets/train.csv')
pd.set_option('display.max_columns', None)
df_house.head(3)

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA,Ex,Y,SBrkr,725,754,0,1479,0.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,1976.0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,GasA,Ex,Y,SBrkr,913,1209,0,2122,1.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1997.0,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,GasA,TA,Y,SBrkr,1057,0,0,1057,1.0,0.0,1,0,3,1,Gd,5,Typ,0,NaN,Detchd,1953.0,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,NaN,NaN,NaN,0,1,2010,WD,109000


In [140]:
df_house.columns = df_house.columns.str.lower().str.replace(' ', '_')

In [141]:
df_house.drop(columns=['alley',
                 'fireplace_qu',
                 'pool_qc',
                 'fence',
                 'misc_feature',
                 'pid'], inplace=True)

In [142]:
df_house = df_house[df_house['gr_liv_area'] < 4000]

In [143]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [144]:
df_nums = df_house.select_dtypes(include=[np.number])
df_nums.head(3)

,id,ms_subclass,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,bsmtfin_sf_1,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,totrms_abvgrd,fireplaces,garage_yr_blt,garage_cars,garage_area,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold,saleprice
0,109,60,NaN,13517,6,8,1976,2005,289.0,533.0,0.0,192.0,725.0,725,754,0,1479,0.0,0.0,2,1,3,1,6,0,1976.0,2.0,475.0,0,44,0,0,0,0,0,3,2010,130500
1,544,60,43.0,11492,7,5,1996,1997,132.0,637.0,0.0,276.0,913.0,913,1209,0,2122,1.0,0.0,2,1,4,1,8,1,1997.0,2.0,559.0,0,74,0,0,0,0,0,4,2009,220000
2,153,20,68.0,7922,5,7,1953,2007,0.0,731.0,0.0,326.0,1057.0,1057,0,0,1057,1.0,0.0,1,0,3,1,5,0,1953.0,1.0,246.0,0,52,0,0,0,0,0,1,2010,109000


In [145]:
df_house.select_dtypes(include=[np.object_]).head(3)

,ms_zoning,street,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_type_2,heating,heating_qc,central_air,electrical,kitchen_qual,functional,garage_type,garage_finish,garage_qual,garage_cond,paved_drive,sale_type
0,RL,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,Gable,CompShg,HdBoard,Plywood,BrkFace,Gd,TA,CBlock,TA,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD
1,RL,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD
2,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,TA,Gd,CBlock,TA,TA,No,GLQ,Unf,GasA,TA,Y,SBrkr,Gd,Typ,Detchd,Unf,TA,TA,Y,WD


In [146]:
df_house.head(3)

,id,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,overall_qual,overall_cond,year_built,year_remod/add,roof_style,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,kitchen_qual,totrms_abvgrd,functional,fireplaces,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,60,RL,NaN,13517,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA,Ex,Y,SBrkr,725,754,0,1479,0.0,0.0,2,1,3,1,Gd,6,Typ,0,Attchd,1976.0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,0,3,2010,WD,130500
1,544,60,RL,43.0,11492,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,GasA,Ex,Y,SBrkr,913,1209,0,2122,1.0,0.0,2,1,4,1,Gd,8,Typ,1,Attchd,1997.0,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,0,4,2009,WD,220000
2,153,20,RL,68.0,7922,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,GasA,TA,Y,SBrkr,1057,0,0,1057,1.0,0.0,1,0,3,1,Gd,5,Typ,0,Detchd,1953.0,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,0,1,2010,WD,109000


In [147]:
df_cat = df_house[['id','neighborhood', 'house_style', 'exter_qual', 'kitchen_qual']]

In [148]:
df_cat.head(3)

,id,neighborhood,house_style,exter_qual,kitchen_qual
0,109,Sawyer,2Story,Gd,Gd
1,544,SawyerW,2Story,Gd,Gd
2,153,NAmes,1Story,TA,Gd


In [149]:
df_combined = pd.merge(left = df_nums, right = df_cat, how = 'inner')

In [150]:
df_combined.head(3)

,id,ms_subclass,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,bsmtfin_sf_1,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,totrms_abvgrd,fireplaces,garage_yr_blt,garage_cars,garage_area,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold,saleprice,neighborhood,house_style,exter_qual,kitchen_qual
0,109,60,NaN,13517,6,8,1976,2005,289.0,533.0,0.0,192.0,725.0,725,754,0,1479,0.0,0.0,2,1,3,1,6,0,1976.0,2.0,475.0,0,44,0,0,0,0,0,3,2010,130500,Sawyer,2Story,Gd,Gd
1,544,60,43.0,11492,7,5,1996,1997,132.0,637.0,0.0,276.0,913.0,913,1209,0,2122,1.0,0.0,2,1,4,1,8,1,1997.0,2.0,559.0,0,74,0,0,0,0,0,4,2009,220000,SawyerW,2Story,Gd,Gd
2,153,20,68.0,7922,5,7,1953,2007,0.0,731.0,0.0,326.0,1057.0,1057,0,0,1057,1.0,0.0,1,0,3,1,5,0,1953.0,1.0,246.0,0,52,0,0,0,0,0,1,2010,109000,NAmes,1Story,TA,Gd


In [151]:
X = df_combined.drop(columns=['id', 'saleprice'])
y = df_combined['saleprice']

In [152]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

In [153]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [154]:
from sklearn.compose import make_column_transformer
ohe = OneHotEncoder(handle_unknown='ignore')
smart_encoder = make_column_transformer((ohe,
                                        ['neighborhood', 'house_style', 'exter_qual', 'kitchen_qual']),
                                        remainder='passthrough',
                                        verbose_feature_names_out=False)

X_train_enc = smart_encoder.fit_transform(X_train)
X_val_enc = smart_encoder.transform(X_val)

X_train_enc = pd.DataFrame(X_train_enc, columns = smart_encoder.get_feature_names_out())
X_val_enc = pd.DataFrame(X_val_enc, columns = smart_encoder.get_feature_names_out())

In [155]:
si = SimpleImputer(strategy = 'constant', fill_value=0)

X_train_fill = si.fit_transform(X_train_enc)
X_val_fill = si.transform(X_val_enc)

In [156]:
X_train_fill = pd.DataFrame(X_train_fill, columns = si.feature_names_in_)
X_val_fill = pd.DataFrame(X_val_fill, columns = si.feature_names_in_)

In [157]:
ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train_fill)
X_val_sc = ss.transform(X_val_fill)

X_train_sc = pd.DataFrame(X_train_sc, columns = ss.get_feature_names_out())
X_val_sc = pd.DataFrame(X_val_sc, columns = ss.get_feature_names_out())

In [158]:
X_train_sc.shape

(1536, 80)

In [159]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(include_bias=False)
X_train_p = poly.fit_transform(X_train_sc)
X_val_p = poly.transform(X_val_sc)

X_train_p = pd.DataFrame(X_train_p, columns = poly.get_feature_names_out())
X_val_p = pd.DataFrame(X_val_p, columns = poly.get_feature_names_out())

In [160]:
X_train_p.shape

(1536, 3320)

In [161]:
from sklearn.linear_model import LassoCV

lasso_cv = LassoCV(cv = 10, n_jobs=-1).fit(X_train_p, y_train)

print('best alpha:', lasso_cv.alpha_)
print('score:', lasso_cv.score(X_train_p, y_train))

best alpha: 940.3895649104886
score: 0.9532818207225963


In [162]:
from sklearn.compose import TransformedTargetRegressor
tt = TransformedTargetRegressor(regressor = lasso_cv, func = np.log, inverse_func = np.exp)
tt.fit(X_train_p, y_train)
tt.score(X_train_p, y_train)

0.9494766747550066

In [163]:
tt.score(X_val_p, y_val)

0.9468757775706345

In [164]:
lasso_cv.score(X_val_p, y_val)

0.9413221049454015

In [165]:
from sklearn.metrics import mean_squared_error
y_train_preds = tt.predict(X_train_p)
mean_squared_error(y_train, y_train_preds)**0.5

17801.61584299862

In [166]:
from sklearn.metrics import mean_squared_error
y_val_preds = tt.predict(X_val_p)
mean_squared_error(y_val, y_val_preds)**0.5

18326.413184358666

---

**CREATING KAGGLE SUBMISSION FILE:**

In [167]:
df_test = pd.read_csv('./datasets/test.csv')

df_test.columns = df_test.columns.str.lower().str.replace(' ', '_')

test_id = df_test['id']

df_test.drop(columns=['alley',
                 'fireplace_qu',
                 'pool_qc',
                 'fence',
                 'misc_feature',
                 'pid'], inplace=True)

df_nums_test = df_test.select_dtypes(include=[np.number])
df_cat_test = df_test[['id', 'neighborhood', 'house_style', 'exter_qual', 'kitchen_qual']]
df_combined_test = pd.merge(left = df_nums_test, right = df_cat_test, how = 'inner')

X = df_combined_test.drop(columns=['id'])

In [168]:
df_test.shape

(878, 74)

In [169]:
df_test.shape

(878, 74)

In [170]:
X.head(3)

,ms_subclass,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,mas_vnr_area,bsmtfin_sf_1,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,1st_flr_sf,2nd_flr_sf,low_qual_fin_sf,gr_liv_area,bsmt_full_bath,bsmt_half_bath,full_bath,half_bath,bedroom_abvgr,kitchen_abvgr,totrms_abvgrd,fireplaces,garage_yr_blt,garage_cars,garage_area,wood_deck_sf,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold,neighborhood,house_style,exter_qual,kitchen_qual
0,190,69.0,9142,6,8,1910,1950,0.0,0,0,1020,1020,908,1020,0,1928,0,0,2,0,4,2,9,0,1910.0,1,440,0,60,112,0,0,0,0,4,2006,OldTown,2Story,TA,Fa
1,90,NaN,9662,5,4,1977,1977,0.0,0,0,1967,1967,1967,0,0,1967,0,0,2,0,6,2,10,0,1977.0,2,580,170,0,0,0,0,0,0,8,2006,Sawyer,1Story,TA,TA
2,60,58.0,17104,7,5,2006,2006,0.0,554,0,100,654,664,832,0,1496,1,0,2,1,3,1,7,1,2006.0,2,426,100,24,0,0,0,0,0,9,2006,Gilbert,2Story,Gd,Gd


In [171]:
X_enc = smart_encoder.transform(X)

X_enc = pd.DataFrame(X_enc, columns = smart_encoder.get_feature_names_out())

In [172]:
X_fill = si.transform(X_enc)

X_fill = pd.DataFrame(X_fill, columns = si.feature_names_in_)

In [173]:
X_sc = ss.transform(X_fill)

X_sc = pd.DataFrame(X_sc, columns = ss.get_feature_names_out())

In [174]:
X_p = poly.transform(X_sc)

X_p = pd.DataFrame(X_p, columns = poly.get_feature_names_out())

In [175]:
preds = tt.predict(X_p)

In [176]:
X_p.shape

(878, 3320)

In [177]:
df_test['saleprice'] = preds

In [178]:
submission = df_test[['id', 'saleprice']]

In [179]:
submission.shape

(878, 2)

In [180]:
submission.head()

,id,saleprice
0,2658,132855.485180
1,2718,151030.384238
2,2414,198926.175347
3,1989,108212.176133
4,625,167899.102857


In [110]:
submission.to_csv('./datasets/lasso_8.csv', index=False)